In [2]:
import numpy as np
import os as os
from geometric_clustering import Geometric_Clustering
import networkx as nx
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import pandas as pd
import pickle
import pylab as plt
from sklearn import metrics
from tqdm.auto import tqdm

# Parameters

In [4]:
NGraphs = 100    # number of realisations
folder = '/data/gosztolai/geocluster/'

method = ['sm', 'm', 'linearized'] #signed modularity, modularity

#Girvan-Newman
p_in = np.round(np.concatenate((np.linspace(0.15,0.34,20),np.linspace(0.36,0.42,4))),2) #edge prob. inside clusters

#Fan
w_in = np.round(np.concatenate((np.linspace(1.0,1.15,7),np.linspace(1.2,1.7,6))),3) #edge weight inside clusters

#LFR
mu = np.round(np.linspace(0.1,0.9,5),1) #mixing

# Functions

In [5]:
def fracCIN(C,D):
    m = int(max(C)+1)
    n = int(max(D)+1)
    Mi = np.zeros(n)
    k = np.zeros(m)
    p = np.zeros(m)
    for i in range(m):
        for j in range(n):
            Mi[j] = len(np.intersect1d(np.where(C==i), np.where(D==j), assume_unique=True))
    
        k[i] = np.argmax(Mi) 
        p[i] = np.max(Mi)
        
    for j in range(n):
        if np.sum(k == j) > 1:
            p[k == j] = 0

    return np.sum(p)/len(C)

def loopOverCases(cases, parname, method):
    N = len(cases)
    m = len(method)
    meannMI = np.zeros((m,N)); stdnMI = np.zeros((m,N)) 
    meanfracCC = np.zeros((m,N)); stdfracCC = np.zeros((m,N))  
    for i in tqdm(range(N)):
        for j, meth in enumerate(method):
            nMI = np.zeros(NGraphs); fracCC = np.zeros(NGraphs)
            stop = 0; k = 0
            while stop == 0 and k < NGraphs:
                
                filename = 'graph_' + str(k) + '_' + parname + '_' + str(cases[i]) + '_cluster_' + meth + '.pkl'
                
                if os.path.exists(filename):
                    stability_results = pickle.load(open(filename,'rb'))
                    labels = np.array(stability_results[1]['community_id'].values) 
                    labels_gt = np.array(stability_results[2])
                    
                    n_t = len(labels)
                    nMItemp = np.zeros(n_t); fracCCtemp = np.zeros(n_t)
                    for l in range(n_t):
                    
                        nMItemp[l] = metrics.normalized_mutual_info_score(labels_gt,labels[l],average_method='arithmetic')
                        fracCCtemp[l] = fracCIN(labels_gt, labels[l])

                    nMI[k] = np.max(nMItemp)
                    fracCC[k] = np.max(fracCCtemp)
                    k += 1
                else:
                    stop = 1

            meannMI[j,i] = np.mean(nMI[nMI!=0])
            stdnMI[j,i] = np.std(nMI[nMI!=0]) 
            meanfracCC[j,i] = np.mean(fracCC[fracCC!=0])
            stdfracCC[j,i] = np.std(fracCC[fracCC!=0]) 
            
    return meannMI, stdnMI, meanfracCC, stdfracCC

def appendthings(cases, parname):
    import pandas as pd
    N = len(cases)
    for i in tqdm(range(N)):
        for j, meth in enumerate(method):
            nMI = np.zeros(NGraphs); fracCC = np.zeros(NGraphs)
            stop = 0; k = 0
            while stop == 0 and k < NGraphs:
                
                filename = 'graph_' + str(k) + '_' + parname + '_' + str(cases[i]) + '_cluster_' + meth + '.pkl'
                
                if os.path.exists(filename):
                
                    stability_results = pickle.load(open(filename,'rb'))
                    stability_results[2] = stability_results[2]['community_ground_truth'].values
                    #labels_gt = np.concatenate((np.zeros(32),np.ones(32),2*np.ones(32),3*np.ones(32)))
                    #labels_gt = []
                    #for l in stability_results[0].nodes:
                    #    print(l)
                    #    labels_gt = np.append(labels_gt,stability_results[0].node[l]['block'])
                    #df = pd.DataFrame({'community_ground_truth' : labels_gt})
                    #stability_results.append(df)
                    pickle.dump(stability_results,open(filename,'wb'))
                    k += 1
                else:
                    stop = 1

# Evaluate benchmarks

In [ ]:
#Girvan-Newman
os.chdir(folder + 'Girvan_Newman')

meannMI_GN, stdnMI_GN, meanfracCC_GN, stdfracCC_GN = loopOverCases(p_in,'p_in',method)
#appendthings(p_in,'p_in')

In [6]:
#Fan
os.chdir(folder + 'Fan')

meannMI_Fan, stdnMI_Fan, meanfracCC_Fan, stdfracCC_Fan = loopOverCases(w_in,'w_in',method)

/home/gosztolai/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/gosztolai/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/gosztolai/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/gosztolai/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/gosztolai/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
#filename = 'graph_' + str(k) + '_' + parname + '_' + str(cases[i]) + '_cluster_' + meth + '.pkl'
#stability_results = pickle.load(open(filename,'rb'))
fname = 'graph_0_w_in_1.0.pkl'
data = pickle.load(open(fname,'rb'))
data[0]

array([-0.00577114, -0.00637482, -0.00701714, -0.00769344, -0.00839663,
       -0.00911676, -0.00984053, -0.01055089, -0.00750116, -0.00820598,
        0.06148376,  0.12145615,  0.21384396,  0.30640997,  0.37075217,
        0.46384822,  0.53932947,  0.60766166,  0.67124835,  0.73215622])

In [ ]:
#LFR
os.chdir(folder + 'LFR')      

meannMI_LFR, stdnMI_LFR, meanfracCC_LFR, stdfracCC_LFR = loopOverCases(mu,'mu',method)

# Plot

In [ ]:
os.chdir(folder + 'Girvan_Newman')
filename = 'graph_' + str(50) + '_p_in_' + str(0.36) + '_cluster_linearized.pkl'
labels = np.array(stability_results[1]['community_id'].values)
print(labels[5])
print(labels[1])

In [ ]:
plt.figure(figsize=(10,12))

#Girvan-Newman
z_out = (0.5-p_in)/3*96 #node degree between clusters
ax0 = plt.subplot(321)
plt.errorbar(z_out/16,meannMI_GN[0,:],stdnMI_GN[0,:],marker='o',c='C0')
ax0.set_xlabel(r'Proportion of between-cluster edges, $z_{out}\,/\,k$')
ax0.set_ylabel("Normalised mutual information", color='C0')
ax0.set_xlim([0.2, 0.8])
ax0.set_ylim([0, 1.1])
ax0.tick_params('y', colors='C0')
ax01 = ax0.twinx()
plt.errorbar(z_out/16,meanfracCC_GN[0,:],stdfracCC_GN[0,:],marker='o',c='C2')
ax01.tick_params('y', colors='C2')
ax01.set_ylabel('Fraction of correctly identified nodes', color='C2')
ax01.yaxis.set_label_position('right')
ax01.set_ylim([0, 1.1])

ax2 = plt.subplot(322)
labels = ['SA', 'Potts', 'ORm', 'MS', 'MOD', 'GN']
zout6 = [0.99, 0.97, meanfracCC_GN[0,16], meanfracCC_GN[1,16], meanfracCC_GN[2,16],  0.9]
zout7 = [0.97, 0.91, meanfracCC_GN[0,13], meanfracCC_GN[1,13], meanfracCC_GN[2,13], 0.67]
zout8 = [0.87, 0.7,  meanfracCC_GN[0,10], meanfracCC_GN[1,10], meanfracCC_GN[2,10], 0.31]

x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

ax2.bar(x - width/3*2, zout6, width, label=r'$z_{out}\,/\,k = 0.38$')
ax2.bar(x , zout7, width, label=r'$z_{out}\,/\,k = 0.44$')
ax2.bar(x + width/3*2, zout8, width, label=r'$z_{out}\,/\,k = 0.5$')
ax2.set_ylabel('Fraction of correctly identified nodes')
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.legend()

#Fan
ax1 = plt.subplot(323)
plt.errorbar(2-w_in,meannMI_Fan[0,:],stdnMI_Fan[0,:],marker='o',c='C0')
ax1.set_xlabel(r'Weight of between-cluster edges, $w_{out}$')
ax1.set_ylabel("Normalised mutual information", color='C0')
ax1.set_xlim([0.5, 1])
ax1.set_ylim([0, 1.1])
ax1.tick_params('y', colors='C0')
ax11 = ax1.twinx()
plt.errorbar(2-w_in,meanfracCC_Fan[0,:],stdfracCC_Fan[0,:],marker='o',c='C2')
ax11.tick_params('y', colors='C2')
ax11.set_ylabel('Fraction of correctly identified nodes', color='C2')
ax11.yaxis.set_label_position('right')
ax11.set_ylim([0, 1.1])

ax3 = plt.subplot(324)
labels = ['Potts', 'ORm', 'MOD', 'WEO', 'GN']
win6 = [1, meanfracCC_Fan[0,6], meanfracCC_Fan[1,6], 0.95, 0.3]
win925 = [0.9, meanfracCC_Fan[0,4], meanfracCC_Fan[1,4], 0.7, 0.05]
win95 = [0.8, meanfracCC_Fan[0,3], meanfracCC_Fan[1,3], 0.6, 0.05]

x = np.arange(len(labels))
ax3.bar(x - width/3*2, win6, width, label=r'$w_{in} = 0.6$')
ax3.bar(x , win925, width, label=r'$w_{in} = 0.95$')
ax3.bar(x + width/3*2, win95, width, label=r'$w_{in} = 0.975$')
ax3.set_ylabel('Fraction of correctly identified nodes')
ax3.set_xticks(x)
ax3.set_xticklabels(labels)
ax3.legend()

#LFR
ax4 = plt.subplot(325)
plt.errorbar(mut,meannMI_LFR[0,:],stdnMI_LFR[0,:],marker='o',c='C0')
ax4.set_xlabel(r'Mixing parameter, $\mu$')
ax4.set_ylabel("Normalised mutual information", color='C0')
ax4.set_xlim([0, 1])
ax4.set_ylim([0, 1.1])
ax4.tick_params('y', colors='C0')
ax41 = ax4.twinx()
plt.errorbar(mut,meanfracCC_LFR[0,:],stdfracCC_LFR[0,:],marker='o',c='C2')
ax41.tick_params('y', colors='C2')
ax41.set_ylabel('Fraction of correctly identified nodes', color='C2')
ax41.yaxis.set_label_position('right')
ax41.set_ylim([0, 1.1])

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.4, hspace=0.25)

MI = np.zeros([NGraphs,n_t])
Stability = np.zeros([NGraphs,n_t])
NComms = np.zeros([NGraphs,n_t])
meanMI = np.zeros([n_t,len(p_in)])
meanStability = np.zeros([n_t,len(p_in)])
meanNComms = np.zeros([n_t,len(p_in)])
stdMI = np.zeros([n_t,len(p_in)])
stdStability = np.zeros([n_t,len(p_in)])
stdNComms = np.zeros([n_t,len(p_in)])
meannMI = np.zeros(len(p_in))     
stdnMI = np.zeros(len(p_in)) 
labels_gt = np.concatenate((np.zeros(32),np.ones(32),2*np.ones(32),3*np.ones(32)))
for i in range(len(p_in)):
    nMI = np.zeros(NGraphs)
    
    for k in range(NGraphs):
        gc.filename = 'graph_' + str(k) + '_p_in_' + str(p_in[i]) + '_cluster.pkl'
        stability_results = pickle.load(open(gc.filename,'rb'))
        MI[k] = np.array(stability_results[1]['MI'].values)
        Stability[k] = np.array(stability_results[1]['stability'].values)
        NComms[k] = np.array(stability_results[1]['number_of_communities'].values)
        #print(stability_results[1]['number_of_communities'].values)
        times = np.log10(stability_results[1]['Markov time'].values)#variation of optimal markov time with pin
        labels = np.array(stability_results[1]['community_id'].values)
        nMItemp = np.zeros(n_t)
        for l in range(n_t):
            nMItemp[l] = metrics.normalized_mutual_info_score(labels_gt,labels[l],average_method='arithmetic') 
        nMI[k] = np.max(nMItemp)
        
    meanMI[:,i] = np.mean(MI,0)   
    meanStability[:,i] = np.mean(Stability,0) 
    meanNComms[:,i] = np.mean(NComms,0)
    stdMI[:,i] = np.std(MI,0)  
    stdStability[:,i] = np.std(Stability,0)
    stdNComms[:,i] = np.std(NComms,0)
    
    meannMI[i] = np.mean(nMI)
    stdnMI[i] = np.std(nMI)

In [ ]:
plt.figure(figsize=(5,5))

gs = gridspec.GridSpec(2, 1, height_ratios = [1., 0.5])
gs.update(hspace=0)
 
labels = []    
ax0 = plt.subplot(gs[0, 0])
for i in range(len(p_in)):
    ax0.plot(times, meanNComms[:,i]) 
    labels.append(str(round(z_out[i],1)))
ax0.yaxis.tick_left()
ax0.yaxis.set_label_position('left')
ax0.axis([times[0],times[-1],0,25])
ax0.set_ylabel('Number of clusters')
ax0.legend(labels,ncol=2,title=r'$z_{out}$')

#plot the number of clusters
#ax1 = ax0.twinx()
#ax1.plot(times, meanNComms,label='size',lw=2.)  
#for i in range(len(p_in)):
#    ax1.plot(times, meanNComms[:,i]-stdNComms[:,i], meanNComms[:,i]+stdNComms[:,i])
#ax1.yaxis.tick_right()
#ax1.tick_params('y', colors='C0')
#ax1.yaxis.set_label_position('right')

    
ax2 = plt.subplot(gs[1, 0])

#first plot the stability
#ax2.plot(times, meanStability[:,0], label=r'$Q$',c='C2')
#ax2.fill_between(times, meanStability[:,0]-stdStability[:,0], meanStability[:,0]+stdStability[:,0])
#ax2.set_yscale('log') 
#ax2.tick_params('y', colors='C2')
#ax2.set_ylabel('Modularity', color='C2')
#ax2.yaxis.set_label_position('left')
#ax2.set_xlabel(r'$log_{10}(t)$')

#plot the MI
#ax3 = ax2.twinx()
#ax3.plot(times, meanMI[:,0],'-',lw=2.,c='C3',label='MI')
for i in range(len(p_in)):
    ax2.plot(times, meanMI[:,0],'-',lw=2.,c='C3',label='MI')
#    ax3.fill_between(times, meanMI[:,i]-stdMI[:,i], meanMI[:,i]+stdMI[:,i])
#ax3.yaxis.tick_right()
#ax3.tick_params('y', colors='C3')
#ax3.set_ylabel(r'Mutual information', color='C3')
#ax3.axhline(1,ls='--',lw=1.,c='C3')
#ax3.axis([times[0], times[-1], 0,1.1])